# Toy exemple - Data Comprehension

## Imports and Required Packages

In [16]:
! pip install geopandas
! pip install fiona==1.9.6
! pip install dash dash-leaflet geopandas pandas

# ! pip install --upgrade geopandas
# ! pip show geopandas
# ! pip show fiona

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached fiona-1.9.6.tar.gz (411 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [3 lines of output]
      <string>:86: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
      CRITICAL:root:A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a sub

In [17]:
import json
import requests
import pandas as pd
import geopandas as gpd

import dash
import dash_leaflet as dl

from dash import html
from time import sleep

In [18]:
df = pd.read_csv("../data/20250401_atividade_economica.csv", encoding="utf-8", sep=';')

In [19]:
abr_logradouro = df["DESC_LOGRADOURO"].dropna().unique()
abr_logradouro = sorted(abr_logradouro)

for v in abr_logradouro:
    print(v)

ALA
AVE
BEC
EST
LRG
PCA
ROD
RUA
TRE
TRV
VDP
VIA


In [20]:
abreviacoes_logradouro = {
    "ALA": "ALA",
    "AVE": "AVENIDA",
    "BEC": "BECO",
    "EST": "ESTRADA",
    "LRG": "LARGO",
    "PCA": "PRAÇA",
    "ROD": "RODOVIA",
    "RUA": "RUA",
    "TRE": "TRECHO",
    "TRV": "TRAVESSA",
    "VDP": "VIADUTO",
    "VIA": "VIA"
}

In [21]:
termos = ["BAR", "BARES" "RESTAURANTE", "RESTAURANTES", "CHOPERIA", "CHOPERIAS", "PUB", "PUBS" "BOTECO", "BOTECOS"]
regex = r'\b(' + '|'.join(termos) + r')\b'
mask = df["DESCRICAO_CNAE_PRINCIPAL"].str.upper().str.contains(regex, na=False, regex=True)
df_new = df[mask]

df_new.head()

/tmp/ipykernel_60417/517599957.py:3: UserWarning:

This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.



,ID_ATIV_ECON_ESTABELECIMENTO,CNAE_PRINCIPAL,DESCRICAO_CNAE_PRINCIPAL,CNAE,DATA_INICIO_ATIVIDADE,NATUREZA_JURIDICA,PORTE_EMPRESA,AREA_UTILIZADA,IND_SIMPLES,IND_MEI,...,FORMA_ATUACAO,DESC_LOGRADOURO,NOME_LOGRADOURO,NUMERO_IMOVEL,COMPLEMENTO,NOME_BAIRRO,NOME,NOME_FANTASIA,CNPJ,GEOMETRIA
17,1023,5611201.0,RESTAURANTES E SIMILARES,5611201,01-07-1993,EMPRESÁRIO (INDIVIDUAL),MICROEMPRESA - ME,82.0,S,S,...,ESTABELECIMENTO FIXO,RUA,DESEMBARGADOR REIS ALVES,90,"ANDAR: 2,",BAIRRO DAS INDUSTRIAS I,APARECIDA MARIA DE SOUZA,NaN,71102990000117,POINT (604468.46 7792708.63)
19,1025,5611201.0,RESTAURANTES E SIMILARES,5611201,15-10-1993,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,125.0,N,N,...,ESTABELECIMENTO FIXO,RUA,LUIZ PONGELUPE,290,C,CARDOSO,LUCAS SILVA DE ARAUJO LTDA,PIZZARIA E CHURRASCARIA VARANDA,71164891000160,POINT (603585.25 7787403.83)
29,1056,5611201.0,RESTAURANTES E SIMILARES,5611201,02-05-1994,EMPRESÁRIO (INDIVIDUAL),MICROEMPRESA - ME,42.0,S,N,...,ESTABELECIMENTO FIXO,AVE,RESSACA,118,LOJA 04,PADRE EUSTAQUIO,PAULO EMILIO COELHO,RABBIT BURGER,97506497000156,POINT (606050.57 7796975.21)
172,1071,5611201.0,RESTAURANTES E SIMILARES,"5611201, 5611203",24-11-1993,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,74.0,S,N,...,ESTABELECIMENTO FIXO,RUA,DESEMBARGADOR RIBEIRO DA LUZ,135,NaN,BARREIRO,GARAPAO BAR E RESTAURANTE LTDA,TATU REI DO ANGU A BAHIANA,71393631000166,POINT (603011.98 7791138.07)
363,705,5611201.0,RESTAURANTES E SIMILARES,"4721103, 4722901, 4723700, 5611201",01-10-1993,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,11.0,S,N,...,ESTABELECIMENTO FIXO,AVE,CRISTIANO MACHADO,1950,LOJA 36,CIDADE NOVA,FRIOS ALMEIDA LTDA,ESPACO DA FEIJOADA,71151047000102,POINT (612160.77 7800283.54)


In [22]:
def expandir_logradouro(desc):
    if pd.isna(desc):
        return ""
    desc = desc.strip().upper()
    return abreviacoes_logradouro.get(desc, desc.title())

def formatar_endereco(row):
    logradouro = expandir_logradouro(row["DESC_LOGRADOURO"])
    nome_logradouro = str(row["NOME_LOGRADOURO"]).strip() if pd.notna(row["NOME_LOGRADOURO"]) else ""
    numero = str(row["NUMERO_IMOVEL"]).strip() if pd.notna(row["NUMERO_IMOVEL"]) else ""
    bairro = str(row["NOME_BAIRRO"]).strip() if pd.notna(row["NOME_BAIRRO"]) else ""

    endereco_principal = ' '.join([p for p in [logradouro, nome_logradouro] if p])

    partes = [endereco_principal]
    if numero:
        partes.append(numero)
    if bairro:
        partes.append(bairro)

    partes += ["Belo Horizonte", "MG", "Brasil"]  

    return ', '.join(partes)

In [23]:
df_new["ENDERECO_COMPLETO"] = df_new.apply(formatar_endereco, axis=1)


df_final = df_new[[
    "ID_ATIV_ECON_ESTABELECIMENTO", "CNAE_PRINCIPAL", "DATA_INICIO_ATIVIDADE",
    "IND_POSSUI_ALVARA", "ENDERECO_COMPLETO", "NOME", "NOME_FANTASIA"
]]

df_final["NOME_FANTASIA"] = df_final["NOME_FANTASIA"].fillna("Estabelecimento sem nome")
# -> ou aqui podemos substituir pelo nome padrão

df_final.to_csv("dados_filtrados.csv", index=False, encoding="utf-8")

/tmp/ipykernel_60417/1314509366.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_60417/1314509366.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
df_final.head()

,ID_ATIV_ECON_ESTABELECIMENTO,CNAE_PRINCIPAL,DATA_INICIO_ATIVIDADE,IND_POSSUI_ALVARA,ENDERECO_COMPLETO,NOME,NOME_FANTASIA
17,1023,5611201.0,01-07-1993,NÃO,"RUA DESEMBARGADOR REIS ALVES, 90, BAIRRO DAS I...",APARECIDA MARIA DE SOUZA,Estabelecimento sem nome
19,1025,5611201.0,15-10-1993,NÃO,"RUA LUIZ PONGELUPE, 290, CARDOSO, Belo Horizon...",LUCAS SILVA DE ARAUJO LTDA,PIZZARIA E CHURRASCARIA VARANDA
29,1056,5611201.0,02-05-1994,NÃO,"AVENIDA RESSACA, 118, PADRE EUSTAQUIO, Belo Ho...",PAULO EMILIO COELHO,RABBIT BURGER
172,1071,5611201.0,24-11-1993,SIM,"RUA DESEMBARGADOR RIBEIRO DA LUZ, 135, BARREIR...",GARAPAO BAR E RESTAURANTE LTDA,TATU REI DO ANGU A BAHIANA
363,705,5611201.0,01-10-1993,SIM,"AVENIDA CRISTIANO MACHADO, 1950, CIDADE NOVA, ...",FRIOS ALMEIDA LTDA,ESPACO DA FEIJOADA


In [25]:
# Teste pequeno para geocodificar com OpenStreetMap
def obter_coordenadas(endereco):
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": endereco,
        "format": "json",
        "limit": 1
    }
    try:
        response = requests.get(url, params=params, headers={"User-Agent": "Mozilla/5.0"})
        data = response.json()
        if data:
            return f"{data[0]['lat']}, {data[0]['lon']}"
    except Exception as e:
        print(f"Erro ao geocodificar: {endereco} - {e}")
    return None


In [26]:
df_final["COORD_GEO"] = ""
coordenadas = []

# processa os 10 primeiros endereços
for i, endereco in enumerate(df_final["ENDERECO_COMPLETO"].head(10)):
    if pd.notna(endereco):
        coord = obter_coordenadas(endereco)
        coordenadas.append(coord)
        print(f"{i+1}. Endereço: {endereco} → Coordenadas: {coord}")
        sleep(1)
    else:
        coordenadas.append("")
        print(f"{i+1}. Endereço vazio → Coordenadas: N/A")

df_final.iloc[:10, df_final.columns.get_loc("COORD_GEO")] = coordenadas

df_final.head(10)


/tmp/ipykernel_60417/1193780154.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



1. Endereço: RUA DESEMBARGADOR REIS ALVES, 90, BAIRRO DAS INDUSTRIAS I, Belo Horizonte, MG, Brasil → Coordenadas: -19.9589334, -44.0017603
2. Endereço: RUA LUIZ PONGELUPE, 290, CARDOSO, Belo Horizonte, MG, Brasil → Coordenadas: -20.0062900, -44.0100157
3. Endereço: AVENIDA RESSACA, 118, PADRE EUSTAQUIO, Belo Horizonte, MG, Brasil → Coordenadas: -19.9177866, -43.9876481
4. Endereço: RUA DESEMBARGADOR RIBEIRO DA LUZ, 135, BARREIRO, Belo Horizonte, MG, Brasil → Coordenadas: -19.9732823, -44.0141195
5. Endereço: AVENIDA CRISTIANO MACHADO, 1950, CIDADE NOVA, Belo Horizonte, MG, Brasil → Coordenadas: -19.8903695, -43.9282191
6. Endereço: RUA LUZIA SALOMAO, 50, MANTIQUEIRA, Belo Horizonte, MG, Brasil → Coordenadas: -19.8027128, -43.9798377
7. Endereço: RUA BRAULIO GOMES NOGUEIRA, 1200, ITAIPU, Belo Horizonte, MG, Brasil → Coordenadas: -19.9945782, -44.0464278
8. Endereço: RUA DOS OTONI, 222, SANTA EFIGENIA, Belo Horizonte, MG, Brasil → Coordenadas: -19.9264845, -43.9245162
9. Endereço: AVENID

,ID_ATIV_ECON_ESTABELECIMENTO,CNAE_PRINCIPAL,DATA_INICIO_ATIVIDADE,IND_POSSUI_ALVARA,ENDERECO_COMPLETO,NOME,NOME_FANTASIA,COORD_GEO
17,1023,5611201.0,01-07-1993,NÃO,"RUA DESEMBARGADOR REIS ALVES, 90, BAIRRO DAS I...",APARECIDA MARIA DE SOUZA,Estabelecimento sem nome,"-19.9589334, -44.0017603"
19,1025,5611201.0,15-10-1993,NÃO,"RUA LUIZ PONGELUPE, 290, CARDOSO, Belo Horizon...",LUCAS SILVA DE ARAUJO LTDA,PIZZARIA E CHURRASCARIA VARANDA,"-20.0062900, -44.0100157"
29,1056,5611201.0,02-05-1994,NÃO,"AVENIDA RESSACA, 118, PADRE EUSTAQUIO, Belo Ho...",PAULO EMILIO COELHO,RABBIT BURGER,"-19.9177866, -43.9876481"
172,1071,5611201.0,24-11-1993,SIM,"RUA DESEMBARGADOR RIBEIRO DA LUZ, 135, BARREIR...",GARAPAO BAR E RESTAURANTE LTDA,TATU REI DO ANGU A BAHIANA,"-19.9732823, -44.0141195"
363,705,5611201.0,01-10-1993,SIM,"AVENIDA CRISTIANO MACHADO, 1950, CIDADE NOVA, ...",FRIOS ALMEIDA LTDA,ESPACO DA FEIJOADA,"-19.8903695, -43.9282191"
549,60382,5611201.0,15-12-2020,NÃO,"RUA LUZIA SALOMAO, 50, MANTIQUEIRA, Belo Horiz...",AECIO CARLOS MIGUEL LEMOS 99648725691,Estabelecimento sem nome,"-19.8027128, -43.9798377"
653,60651,5611201.0,16-09-2011,SIM,"RUA BRAULIO GOMES NOGUEIRA, 1200, ITAIPU, Belo...",CHURRASCARIA BAHIAS BAR LTDA,Estabelecimento sem nome,"-19.9945782, -44.0464278"
662,60660,5611201.0,16-09-2011,SIM,"RUA DOS OTONI, 222, SANTA EFIGENIA, Belo Horiz...",D'MARCUS & CESAR RESTAURANTE LTDA,RESTAURANTE NOVO SABOR,"-19.9264845, -43.9245162"
690,60996,5611201.0,27-09-2011,SIM,"AVENIDA SOLFERINA RICCI PACE, 1301, CONJUNTO J...","DU DAY RESTAURANTE, LANCHONETE E DISTRIBUIDORA...",Estabelecimento sem nome,None
852,61013,5611201.0,30-09-2011,NÃO,"RUA SALINAS, 1547, SANTA TEREZA, Belo Horizont...",SHIRLEY FRAGUAS MESQUITA 85413178620,Estabelecimento sem nome,"-19.9148029, -43.9230373"


In [27]:
df_final.head(10).to_csv("toy_data/enderecos_com_coordenadas_toy.csv", index=False, encoding="utf-8")

In [28]:
gdf = gpd.read_file("../data/BAIRRO_OFICIAL/BAIRRO_OFICIAL.shp")
gdf.to_file("../data/BAIRRO_OFICIAL_bh.geojson", driver="GeoJSON")

In [29]:
df = pd.read_csv("toy_data/enderecos_com_coordenadas_toy.csv")

with open("../data/BAIRRO_OFICIAL_bh.geojson", encoding='utf-8') as f:
    BAIRRO_OFICIAL_geojson = json.load(f)

marcadores = [
    dl.Marker(
        position=[float(lat), float(lon)],
        children=dl.Tooltip(nome)
    )
    for latlon, nome in zip(df["COORD_GEO"], df["NOME_FANTASIA"])
    if isinstance(latlon, str) and ',' in latlon
    for lat, lon in [latlon.split(",")]
]

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Mapa Interativo de Bares em BH"),
    dl.Map(center=[-19.92, -43.94], zoom=12, children=[
        dl.TileLayer(),
        dl.GeoJSON(data=BAIRRO_OFICIAL_geojson),
        dl.LayerGroup(marcadores)
    ], style={'width': '100%', 'height': '80vh'})
])

if __name__ == '__main__':
    app.run(debug=True)
